# Importy

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

# Pobranie i selekcja danych

In [2]:
dataset = pd.read_excel('2014_PM2.5_1g.xlsx', index_col=0)
dataset.head()

,DsWrocWisA,KpAirpWiktorowo,KpBydgPlPozn,KpBydgWarszawska,KpTorunDziewulsk,LdLodzWIOSACzernik,LdZgierzWIOSAMielcza,LuZgoraWIOS_AUT,MpKrakowWIOSAKra6117,MpKrakowWIOSBuja6119,...,SkMalogCemen3,SkPolanRuszcz,SlGliwiGliw_mewy,SlKatowKato_kossu,SlZlotyJano_lesni,WmOlsztyWIOS_Puszkin,WpWKP001,WpWKPKalisz007/10,ZpSzczecin001,ZpSzczecin002
Kod stacji,,,,,,,,,,,,,,,,,,,,,
Wskaźnik,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5,...,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5,PM2.5
Czas uśredniania,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,...,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g
2014-01-01 01:00:00,152,104.56,116.214,140.857,52.4172,81.0293,119.696,NaN,104,78,...,34.12,56.54,188,104,32,84.1216,56.1,NaN,105.489,109
2014-01-01 02:00:00,137,94.3,NaN,121.63,NaN,210.365,70.3375,NaN,104,65,...,34.61,35.93,130,107,39,75.4444,55.2,82.5,149.797,118
2014-01-01 03:00:00,129,102.48,NaN,59.189,46.5652,66.4258,40.9157,NaN,91,63,...,33.58,34.36,138,114,43,34.4275,49.9,83,86.9602,120


In [3]:
def select_columns_by_name(df, name):
    idxs = pd.Series(df.columns).str.match(name)
    idxs = idxs[idxs == True].index.sort_values()
    return df.iloc[:, idxs]

waw = select_columns_by_name(dataset, 'MzWar')

In [4]:
waw.head()

,MzWarNiepodKom,MzWarszUrsynow
Kod stacji,,
Wskaźnik,PM2.5,PM2.5
Czas uśredniania,1g,1g
2014-01-01 01:00:00,100.326,63.49
2014-01-01 02:00:00,69.4124,36.28
2014-01-01 03:00:00,40.9679,30.77


In [5]:
waw = waw.drop("Wskaźnik", axis=0)
waw = waw.drop("Czas uśredniania", axis=0)


In [6]:
for col in waw.select_dtypes(include=[object]).columns:
    waw[col] = waw[col].replace(',', '.', regex=True).astype(float)

In [7]:
waw.head()

,MzWarNiepodKom,MzWarszUrsynow
Kod stacji,,
2014-01-01 01:00:00,100.325798,63.49
2014-01-01 02:00:00,69.412430,36.28
2014-01-01 03:00:00,40.967861,30.77
2014-01-01 04:00:00,37.308937,31.69
2014-01-01 05:00:00,38.185955,36.86


# Czy są puste wartości?

In [8]:
waw.isnull().sum() # Są...

MzWarNiepodKom    616
MzWarszUrsynow    179
dtype: int64

# To może najprostsza metoda i strategia z pandas?

In [9]:
waw_interpol = waw.interpolate(method='linear', axis=1, limit_direction='both')

# Działa? Czy dalej są puste wartości?

In [10]:
waw_interpol.isnull().sum()  # Są...

MzWarNiepodKom    26
MzWarszUrsynow    26
dtype: int64

# Co się stało?

In [11]:
waw_interpol[waw_interpol.isnull().any(axis=1)].head(10)

,MzWarNiepodKom,MzWarszUrsynow
Kod stacji,,
2014-08-14 08:00:00,NaN,NaN
2014-08-14 13:00:00,NaN,NaN
2014-08-18 11:00:00,NaN,NaN
2014-08-18 12:00:00,NaN,NaN
2014-08-18 13:00:00,NaN,NaN
2014-08-18 14:00:00,NaN,NaN
2014-08-18 15:00:00,NaN,NaN
2014-08-18 16:00:00,NaN,NaN
2014-08-18 17:00:00,NaN,NaN


#### To podejście zawodzi, gdy są duże "luki" (tj. sąsiadujące ze sobą puste komórki) w zbiorze danych!

# Wypełnienie średnią z kolumny / medianą

In [12]:
from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer( strategy='mean') # jeśli chcemy medianę, to 'median'
imp_mean.fit(waw)
imputed = imp_mean.transform(waw)

In [13]:
sum(np.isnan(imputed)) # jak widać, działa!

array([0, 0])

#### Jest to proste, szybkie i dobrze działa na małych zbiorach danych - a taki właśnie tu mamy. Uwaga, nie wolno stosować do zbiorów, gdzie mamy zakodowane dane kategoryczne, bo 'dummy variables', oczywiście, to wszystko wyłożą. Dla danych kategorycznych analogicznym podejściem jest strategia 'most_frequent', która interpoluje najczęściej występującymi wartościami.

In [17]:
df_imputed = pd.DataFrame(imputed, columns=['MzWarNiepodKom', 'MzWarszUrsynow'], index=waw_interpol.index)

In [18]:
df_imputed.to_csv('filled.csv')